<a href="https://colab.research.google.com/github/CodewithTanzeel/LangChain_gemini_Calculator-NLP-/blob/main/Calculater_using_toolcalling_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -U -q "google-generativeai>=0.7.2"

In [12]:
!pip install langchain-google-genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


In [3]:
import os
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

In [20]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType, Tool

# Define Calculator class
class Calculator:
    def calculate(self, expression: str) -> str:
        try:
            # Use (Python's eval) to compute the safe result
            result = eval(expression, {"__builtins__": None}, {})
            return str(result)
        except Exception as e:
            return f"Error: {e}"

# Create an instance of Calculator
calculator = Calculator()

# Define calculator as a tool
calculator_tool = Tool(
    name="Calculator",
    func=calculator.calculate,
    description="A simple calculator that evaluates mathematical expressions."
)

# Initialize Gemini model
gemini_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", api_key=userdata.get("GOOGLE_API_KEY")
)

# Define tools
tools = [calculator_tool]

# Initialize agent
agent_executor = initialize_agent(
    tools=tools,
    llm=gemini_model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [23]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Set up memory for conversation context
memory = ConversationBufferMemory(memory_key="chat_history")

# Define a prompt template
prompt = PromptTemplate(
    input_variables=["chat_history", "input"],
    template="Previous conversation: {chat_history}\nUser: {input}\nAI:",
)

# Create an LLM chain with memory
chain = LLMChain(
    llm=gemini_model,  # Assigning the Gemini model
    prompt=prompt,      # Attaching the prompt template
    memory=memory       # Attaching memory for conversation context
)


<ipython-input-23-b6e08076a9b6>:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [25]:
# User query
query = "What is 20 divided by 5?"
# gets response from the system
response = chain.run(query)
print(response)

4


In [26]:

queries = [
    "What is 50 divide by 5?",
    "Now divide the result by 5.",
    "Add 49 to that."
]

for q in queries:
    print("Query:", q)
    print("Response:", chain.run(q))
    print("-" * 40)


Query: What is 50 divide by 5?
Response: 10
----------------------------------------
Query: Now divide the result by 5.
Response: 2
----------------------------------------
Query: Add 49 to that.
Response: 51
----------------------------------------
